In [1]:
import pandas as pd
import numpy as np
from math import exp

import scipy

import datetime
import calendar


In [2]:
#Global weights for scoring (comme pas de référence, on met des valeurs arbitraires au jugement)
WEIGHTS_MANV = {5:4.0, 6:3.5, 10:2.0, 11:2.0, 12:2.0, 17:2.5, 18:2.5, 19:3.0, 21:1.0}
WEIGHTS_COL  = {1:2.0, 2:2.0, 3:1.0, 4:1.0, 5:1.5, 7:-0.5}
WEIGHTS_CHOC = {1:0.5, 4:1.0, 7:0.5, 8:0.5, 9:1.5}
WEIGHTS_OBS  = {0:0.0, 2:1.0, 3:1.0, 4:1.0, 6:1.0, 13:1.0, 16:0.5}
WEIGHTS_OBSM = {0:0.0, 1:1.5, 2:0.5}
WEIGHTS_CATR = {1:0.5, 2:0.3, 3:0.5, 4:0.0}  
WEIGHTS_LUM  = {1:0.0, 2:0.2, 3:0.5, 4:0.4, 5:0.3}
WEIGHTS_AGG  = {1:0.3, 2:0.0}  

#GLOBAL TOPAY 
COUT_CORPO = {"leger": 1500, "grave": 120000, "deces": 200000}

COUT_MAT_CATV = {
    1: 2200,
    2: 1200, 3: 1500,
    7: 3000,
    10: 15000, 11: 15000, 12: 15000, 13: 15000, 14: 15000, 15: 15000, 16: 15000,
    30: 20000, 31: 20000, 32: 20000, 33: 20000,
    35: 10000, 36: 10000, 37: 10000, 38: 10000,
    50: 5000, 60: 5000,
    80: 2500, 99: 2500
}
DEFAULT_COUT_MAT_CATV = 2000

LUM_CORP_MULT = {1: 1.00, 2: 1.10, 3: 1.20, 4: 1.25, 5: 1.05}
AGG_CORP_MULT = {1: 1.15, 2: 1.00}
CATR_CORP_MULT = {1: 1.15, 2: 1.10, 3: 1.05, 4: 1.00}
VMA_CORP_MULT = [(30, 0.85), (50, 1.00), (80, 1.10), (100, 1.25), (999, 1.40)]
MANV_CORP_MULT = {5: 1.35, 6: 1.25, 10: 1.15, 11: 1.15, 12: 1.15, 17: 1.20, 18: 1.20, 19: 1.25, 21: 1.10}
OBSM_CORP_MULT = {1: 1.50, 2: 1.05, 4: 1.05, 5: 1.10}
OBS_CORP_MULT = {2: 1.10, 3: 1.10, 4: 1.10, 6: 1.10, 13: 1.05, 16: 1.05}
PEDESTRIAN_CORP_BONUS = 1.25

COL_MAT_MULT = {1: 1.30, 2: 1.05, 3: 1.15, 4: 1.50, 5: 1.40, 6: 1.00, 7: 0.40}
CHOC_MAT_MULT = {1: 1.13, 4: 0.75, 7: 0.88, 8: 0.88, 9: 1.11}
VMA_MAT_MULT = [(30, 0.85), (50, 1.00), (80, 1.10), (100, 1.20), (999, 1.30)]
AGG_MAT_MULT = {1: 0.95, 2: 1.05}

CATV_CORP_VULN_BONUS = {2: 1.15, 3: 1.15, 21: 1.20, 22: 1.20}


In [3]:
def samedis_annee(annee: int):
    samedis = []
    for mois in range(1, 13):
        nb_jours = calendar.monthrange(annee, mois)[1]
        for jour in range(1, nb_jours + 1):
            date = datetime.date(annee, mois, jour)
            if date.weekday() == 5:  
                samedis.append([jour, mois])
    return samedis

Sam_data = pd.DataFrame(samedis_annee(2023),columns=['jour','mois'])


In [4]:
Sam_data

,jour,mois
0,7,1
1,14,1
2,21,1
3,28,1
4,4,2
5,11,2
6,18,2
7,25,2
8,4,3
9,11,3


In [5]:
Lieux = pd.read_csv(r'dataset_base\lieux-2023.csv', sep=';',low_memory=False)

In [6]:
Caract = pd.read_csv(r'dataset_base\caract-2023.csv', sep=';',low_memory=False)

In [7]:
Caract

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,202300000001,7,5,2023,06:00,1,75,75101,2,4,2,7,RUE DE RIVOLI,"48,86638600","2,32347100"
1,202300000002,7,5,2023,05:30,5,94,94080,2,1,3,6,Avenue de Paris,"48,84547782","2,42868146"
2,202300000003,7,5,2023,20:50,1,94,94022,2,3,2,1,Avenue du Général Leclerc,"48,76240000","2,40655000"
3,202300000004,6,5,2023,23:57,5,94,94078,2,1,3,5,Rue de Paris,"48,73248432","2,44687575"
4,202300000005,7,5,2023,00:50,5,94,94068,2,2,3,3,56bis Avenue Raspail,"48,78581000","2,49217000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54817,202300054818,26,10,2023,20:45,5,974,97416,2,1,1,6,LA FONTAINE (RUE JEAN DE),"-21,33828000","55,47771000"
54818,202300054819,26,10,2023,19:10,3,974,97416,1,1,1,3,RN3 (ANCIENNE ROUTE),"-21,28865000","55,50994000"
54819,202300054820,26,10,2023,09:40,1,974,97411,2,1,1,7,BAMBOU (CHEMIN DE LA RUELLE),"-20,90129000","55,40598000"
54820,202300054821,26,10,2023,17:20,1,973,97302,1,6,1,3,ROUTE NATIONALE 1,"4,89713000","-52,32854000"


In [8]:
Lieux 

,Num_Acc,catr,voie,v1,v2,circ,nbv,vosp,prof,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma
0,202300000001,4,RUE DE RIVOLI,0,NaN,1,2,0,1,-1,-1,1,NaN,-1,2,0,1,30
1,202300000001,4,RUE SAINT FLORENTIN,0,NaN,1,1,0,1,-1,-1,1,NaN,-1,2,0,1,30
2,202300000002,3,120,0,NaN,2,3,2,1,-1,-1,1,NaN,-1,2,0,1,50
3,202300000003,3,5,0,NaN,2,4,0,1,1,0,1,NaN,-1,2,5,1,50
4,202300000003,3,87,0,NaN,2,4,0,1,1,0,1,NaN,-1,2,5,1,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70855,202300054818,4,LA FONTAINE (RUE JEAN DE),0,NaN,2,2,0,1,-1,0,1,NaN,-1,1,0,1,50
70856,202300054819,4,RN3 (ANCIENNE ROUTE),0,NaN,2,2,0,2,0,0,1,NaN,-1,1,0,1,50
70857,202300054820,4,BAMBOU (CHEMIN DE LA RUELLE),0,NaN,2,2,0,2,0,0,2,NaN,-1,1,0,8,30
70858,202300054821,2,ROUTE NATIONALE 1,0,NaN,-1,2,1,1,4,50,1,NaN,-1,1,0,1,90


In [9]:
Samedi=Caract.merge(Sam_data, on=["jour","mois"], how="inner").sort_values(by=["mois","jour"]).reset_index(drop=True)

In [10]:
Samedi

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,202300005200,7,1,2023,16:04,1,44,44109,2,1,1,6,1 Boulevard Général de Gaulle,"47,20738000","-1,54007238"
1,202300011274,7,1,2023,13:20,1,74,74010,2,1,1,1,NANFRAY (ROUTE DE),"45,89410000","6,08803000"
2,202300019151,7,1,2023,19:30,3,13,13077,1,1,1,4,RN 568,"43,41367350","5,00434666"
3,202300021478,7,1,2023,16:56,1,36,36044,2,1,1,7,avenue jacques chirac,"46,81514000","1,68482000"
4,202300031190,7,1,2023,17:27,2,92,92073,2,1,2,2,JAURES (AVENUE JEAN),"48,86453390","2,20746696"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7756,202300010831,30,12,2023,03:05,5,38,38382,1,1,8,6,a.48 pk 090.500 direction lyon,"45,24200400","5,65903200"
7757,202300010850,30,12,2023,11:15,1,973,97305,1,1,1,3,Route Nationale 1,"4,99159500","-52,44990000"
7758,202300010866,30,12,2023,03:15,3,77,77153,1,1,2,5,N2,"49,05439000","2,69953100"
7759,202300010874,30,12,2023,05:40,3,04,04106,1,3,1,3,D4096,"43,96116200","5,89991000"


In [11]:
Samedi_lieux = Samedi.merge(Lieux, on="Num_Acc", how="inner")

In [12]:
Samedi_lieux_nuits= Samedi_lieux[Samedi_lieux["lum"].isin([3, 4, 5])]

In [13]:
Samedi_lieux[Samedi_lieux["lum"].isin([3, 4, 5])]["catr"].map({
    1: "Autoroute",
    2: "Route nationale",
    3: "Route départementale",
    4: "Voie communale",
    5: "Hors réseau public",
    6: "Parc de stationnement ouvert à la circulation publique",
    7: "Routes de métropole urbaine",
    9: "Autre"
}).value_counts(normalize=True) * 100


catr
Voie communale                                            42.350204
Route départementale                                      37.870855
Autoroute                                                  8.958697
Route nationale                                            7.591623
Routes de métropole urbaine                                2.472368
Parc de stationnement ouvert à la circulation publique     0.494474
Autre                                                      0.261780
Name: proportion, dtype: float64

##1 Relation age / accidents
##2 Sans permis

In [14]:
usagers = pd.read_csv(r'dataset_base\usagers-2023.csv', sep=';', low_memory=False)
vehicules = pd.read_csv(r'dataset_base\vehicules-2023.csv', sep=';', low_memory=False)
caract = pd.read_csv(r'dataset_base\caract-2023.csv', sep=';', low_memory=False)
lieux = pd.read_csv(r'dataset_base\lieux-2023.csv', sep=';', low_memory=False)

merged_data = pd.merge(
    usagers,
    vehicules,
    on=['Num_Acc', 'id_vehicule', 'num_veh'],
    suffixes=('_usager', '_vehicule')
)

merged_data = pd.merge(merged_data, caract, on='Num_Acc')
merged_data = pd.merge(merged_data, lieux, on='Num_Acc')


In [15]:
merged_data.drop_duplicates(subset=["Num_Acc","id_usager"]).to_csv('merged_data2023.csv', index=False)

In [16]:
f2023=pd.read_csv('merged_data2023.csv')

C:\Users\moham\AppData\Local\Temp\ipykernel_92504\2276383783.py:1: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  f2023=pd.read_csv('merged_data2023.csv')


In [17]:
f2023['age'] = f2023['an'] - f2023['an_nais']
accidents_cibles = f2023.query("catu == 1 and age >= 20 and age <= 25")['Num_Acc'].unique()

result20_25 = f2023[f2023['Num_Acc'].isin(accidents_cibles)].reset_index()

In [18]:
result20_25

,index,Num_Acc,id_usager,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,...,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma,age
0,10,202300000005,203 851 173,155 680 549,B01,1,1,4,2,2001.0,...,-1,-1,1,NaN,-1,2,0,1,30,22.0
1,11,202300000005,203 851 174,155 680 550,A01,1,1,1,1,2001.0,...,-1,-1,1,NaN,-1,2,0,1,30,22.0
2,21,202300000009,203 851 162,155 680 541,A01,1,1,1,1,1988.0,...,-1,-1,1,NaN,-1,2,5,1,50,35.0
3,22,202300000009,203 851 163,155 680 542,B01,1,1,4,1,2003.0,...,-1,-1,1,NaN,-1,2,5,1,50,20.0
4,23,202300000010,203 851 160,155 680 540,B01,1,1,1,1,1999.0,...,-1,-1,1,NaN,-1,2,0,1,50,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35663,125780,202300054819,203 720 724,155 583 349,A01,1,1,3,1,2003.0,...,0,0,1,NaN,-1,1,0,1,50,20.0
35664,125781,202300054819,203 720 725,155 583 350,B01,1,1,1,2,1982.0,...,0,0,1,NaN,-1,1,0,1,50,41.0
35665,125786,202300054822,203 720 717,155 583 344,A01,2,2,1,2,2003.0,...,-1,-1,1,NaN,-1,1,0,5,30,20.0
35666,125787,202300054822,203 720 718,155 583 344,A01,1,1,1,2,2002.0,...,-1,-1,1,NaN,-1,1,0,5,30,21.0


In [19]:
result20_25.reset_index(drop=True, inplace=True)

In [20]:
result20_25 = result20_25.drop(result20_25.columns[[0]], axis=1)

In [21]:
result20_25

,Num_Acc,id_usager,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,...,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma,age
0,202300000005,203 851 173,155 680 549,B01,1,1,4,2,2001.0,0,...,-1,-1,1,NaN,-1,2,0,1,30,22.0
1,202300000005,203 851 174,155 680 550,A01,1,1,1,1,2001.0,3,...,-1,-1,1,NaN,-1,2,0,1,30,22.0
2,202300000009,203 851 162,155 680 541,A01,1,1,1,1,1988.0,0,...,-1,-1,1,NaN,-1,2,5,1,50,35.0
3,202300000009,203 851 163,155 680 542,B01,1,1,4,1,2003.0,0,...,-1,-1,1,NaN,-1,2,5,1,50,20.0
4,202300000010,203 851 160,155 680 540,B01,1,1,1,1,1999.0,5,...,-1,-1,1,NaN,-1,2,0,1,50,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35663,202300054819,203 720 724,155 583 349,A01,1,1,3,1,2003.0,0,...,0,0,1,NaN,-1,1,0,1,50,20.0
35664,202300054819,203 720 725,155 583 350,B01,1,1,1,2,1982.0,1,...,0,0,1,NaN,-1,1,0,1,50,41.0
35665,202300054822,203 720 717,155 583 344,A01,2,2,1,2,2003.0,2,...,-1,-1,1,NaN,-1,1,0,5,30,20.0
35666,202300054822,203 720 718,155 583 344,A01,1,1,1,2,2002.0,1,...,-1,-1,1,NaN,-1,1,0,5,30,21.0


In [22]:
result20_25.drop_duplicates().to_csv('result20_25.csv', index=False)

In [23]:
def logistic(z, a=-3.0, b=0.6):
    x = a + b*z
    try:
        return 1.0 / (1.0 + exp(-x))
    except OverflowError:
        return 0.0 if x < 0 else 1.0

def score_ligne_conducteur(row):
    score, reasons = 0.0, []

    def add(part, code, weights):
        nonlocal score, reasons
        try:
            code = int(code)
        except Exception:
            return
        w = weights.get(code, 0.0)
        score += w
        if w:
            reasons.append(f"{part}:{code}(+{w})")

    add("manv", row.get("manv", 0), WEIGHTS_MANV)
    add("col",  row.get("col", 0),  WEIGHTS_COL)
    add("choc", row.get("choc", 0), WEIGHTS_CHOC)
    add("obs",  row.get("obs", 0),  WEIGHTS_OBS)
    add("obsm", row.get("obsm", 0), WEIGHTS_OBSM)
    add("catr", row.get("catr", 0), WEIGHTS_CATR)

    try:
        vma = float(row.get("vma", 0))
    except Exception:
        vma = 0.0
    if 0 <= vma <= 50: w = 0.2
    elif vma <= 80:    w = 0.4
    elif vma <= 100:   w = 0.8
    else:              w = 1.2
    score += w
    reasons.append(f"vma:{int(vma)}(+{w})")

    add("lum", row.get("lum", 0), WEIGHTS_LUM)
    add("agg", row.get("agg", 0), WEIGHTS_AGG)

    try:
        if int(row.get("col", 0)) == 1 and int(row.get("manv", 0)) == 5:
            score += 1.5
            reasons.append("rule:frontale+contresens(+1.5)")
    except Exception:
        pass

    return score, reasons

def _sniff_sep(path):
    with open(path, "rb") as f:
        head = f.read(4096)
    try:
        txt = head.decode("utf-8", errors="ignore")
    except:
        txt = head.decode("latin-1", errors="ignore")
    return ";" if txt.count(";") > txt.count(",") else ","

def _read_baac(path):
    sep = _sniff_sep(path)
    try:
        df = pd.read_csv(path, sep=sep, low_memory=False, encoding="utf-8")
    except UnicodeDecodeError:
        df = pd.read_csv(path, sep=sep, low_memory=False, encoding="latin-1")
    df.columns = [c.strip().replace(" ", "_").replace("\xa0","_") for c in df.columns]
    for col in ["Num_Acc","id_usager","id_vehicule","catu","manv","col","choc","obs","obsm",
                "catr","vma","lum","agg","an","an_nais","age"]:
        if col in df.columns and df[col].dtype == object:
            df[col] = (df[col].astype(str)
                               .str.replace("\xa0","", regex=False)
                               .str.strip()
                               .str.replace(",", ".", regex=False))
    return df

def _to_numeric(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

def compute_young_driver_responsability_strict(
    baac_csv: str,
    out_csv: str = "p_resp_young_drivers_per_accident.csv",
    out_csv_drivers: str = "p_resp_par_conducteur.csv",
    age_min: int = 20,
    age_max: int = 25
) -> pd.DataFrame:

    df = _read_baac(baac_csv)
    _to_numeric(df, ["catu","manv","col","choc","obs","obsm","catr","vma","lum","agg","an","an_nais"])

    drivers = df.copy()
    drivers["catu_num"] = pd.to_numeric(drivers.get("catu", np.nan), errors="coerce")
    drivers = drivers[drivers["catu_num"] == 1].copy()
    if drivers.empty:
        pd.DataFrame(columns=["Num_Acc"]).to_csv(out_csv, index=False)
        drivers.to_csv(out_csv_drivers, index=False)
        return pd.DataFrame()

    if "age" in drivers.columns:
        age_series = pd.to_numeric(drivers["age"], errors="coerce")
        if age_series.notna().sum() < max(1, int(0.1*len(drivers))):
            age_series = pd.to_numeric(drivers.get("an", np.nan), errors="coerce") - pd.to_numeric(drivers.get("an_nais", np.nan), errors="coerce")
    else:
        age_series = pd.to_numeric(drivers.get("an", np.nan), errors="coerce") - pd.to_numeric(drivers.get("an_nais", np.nan), errors="coerce")
    drivers["age_calc"] = age_series
    drivers["jeune_20_25"] = (drivers["age_calc"] >= age_min) & (drivers["age_calc"] <= age_max)

    scores, reasons, p_raw = [], [], []
    for _, r in drivers.iterrows():
        s, rs = score_ligne_conducteur(r)
        scores.append(round(s, 3))
        reasons.append(";".join(rs))
        p_raw.append(max(1e-6, logistic(s)))  
    drivers["score_raw"]    = scores
    drivers["reason_codes"] = reasons
    drivers["p_raw"]        = p_raw

    drivers["Num_Acc"] = drivers["Num_Acc"].astype(str).str.strip()
    sum_raw = drivers.groupby("Num_Acc")["p_raw"].transform("sum")
    equal_share = 1.0 / drivers.groupby("Num_Acc")["p_raw"].transform("count")
    drivers["resp_share"] = np.where(sum_raw > 0, drivers["p_raw"] / sum_raw, equal_share)

    eligible = drivers.loc[drivers["jeune_20_25"], "Num_Acc"].dropna().unique()
    if len(eligible) == 0:
        cols = ["Num_Acc","nb_conducteurs_total","nb_conducteurs_jeunes","pct_conducteurs_jeunes",
                "resp_share_jeunes_total","resp_share_autres_total",
                "resp_share_total_check","p_resp_moyenne_jeunes","resp_share_jeune_1","reason_jeune_1","age_jeune_1"]
        pd.DataFrame(columns=cols).to_csv(out_csv, index=False)
        drivers.head(0).to_csv(out_csv_drivers, index=False)
        return pd.DataFrame(columns=cols)

    d_all   = drivers[drivers["Num_Acc"].isin(eligible)].copy()
    d_young = d_all[d_all["jeune_20_25"]].copy()

    cols_front = ["Num_Acc","id_usager","id_vehicule"] if "id_usager" in d_all.columns or "id_vehicule" in d_all.columns else ["Num_Acc"]
    driver_export_cols = cols_front + [c for c in d_all.columns if c not in cols_front] + \
                         ["score_raw","p_raw","resp_share","reason_codes","age_calc","jeune_20_25"]
    driver_export_cols = list(dict.fromkeys(driver_export_cols))
    d_all[driver_export_cols].to_csv(out_csv_drivers, index=False, encoding="utf-8")

    key_for_count = "id_usager" if "id_usager" in d_all.columns else ("id_vehicule" if "id_vehicule" in d_all.columns else None)
    if key_for_count is None:
        d_all["__row_id__"] = np.arange(len(d_all))
        d_young["__row_id__"] = np.arange(len(d_young))
        key_for_count = "__row_id__"

    total_drivers = d_all.groupby("Num_Acc")[key_for_count].nunique().rename("nb_conducteurs_total").reset_index()
    young_counts  = d_young.groupby("Num_Acc")[key_for_count].nunique().rename("nb_conducteurs_jeunes").reset_index()

    share_young = d_young.groupby("Num_Acc")["resp_share"].sum().rename("resp_share_jeunes_total").reset_index()
    share_all   = d_all.groupby("Num_Acc")["resp_share"].sum().rename("resp_share_total_check").reset_index()

    rows = []
    for num_acc, g in d_young.groupby("Num_Acc"):
        g = g.sort_values("resp_share", ascending=False)
        data = {
            "Num_Acc": num_acc,
            "p_resp_moyenne_jeunes": g["p_raw"].mean(),
        }
        for i, (_, row) in enumerate(g.iterrows(), start=1):
            data[f"resp_share_jeune_{i}"] = round(float(row["resp_share"]), 6)
            data[f"reason_jeune_{i}"]     = row["reason_codes"]
            data[f"age_jeune_{i}"]        = row["age_calc"]
        rows.append(data)
    wide = pd.DataFrame(rows)

    out = (wide
           .merge(total_drivers, on="Num_Acc", how="left")
           .merge(young_counts,  on="Num_Acc", how="left")
           .merge(share_young,   on="Num_Acc", how="left")
           .merge(share_all,     on="Num_Acc", how="left"))

    out["nb_conducteurs_total"]  = out["nb_conducteurs_total"].fillna(0).astype(int)
    out["nb_conducteurs_jeunes"] = out["nb_conducteurs_jeunes"].fillna(0).astype(int)
    out["pct_conducteurs_jeunes"] = np.where(
        out["nb_conducteurs_total"] > 0,
        out["nb_conducteurs_jeunes"] / out["nb_conducteurs_total"],
        np.nan
    )
    out["resp_share_jeunes_total"] = out["resp_share_jeunes_total"].clip(lower=0, upper=1)
    out["resp_share_autres_total"] = (1 - out["resp_share_jeunes_total"]).clip(lower=0, upper=1)
    out["resp_share_total_check"]  = out["resp_share_total_check"].round(6)

    cols_base = [
        "Num_Acc","nb_conducteurs_total","nb_conducteurs_jeunes","pct_conducteurs_jeunes",
        "resp_share_jeunes_total","resp_share_autres_total","resp_share_total_check",
        "p_resp_moyenne_jeunes"
    ]
    j_cols = sorted([c for c in out.columns if c.startswith("resp_share_jeune_")], key=lambda x: int(x.split("_")[-1]))
    r_cols = sorted([c for c in out.columns if c.startswith("reason_jeune_")],    key=lambda x: int(x.split("_")[-1]))
    a_cols = sorted([c for c in out.columns if c.startswith("age_jeune_")],       key=lambda x: int(x.split("_")[-1]))
    final = cols_base[:]
    for k in range(1, max([int(c.split("_")[-1]) for c in j_cols] + [0]) + 1):
        if f"resp_share_jeune_{k}" in out.columns: final.append(f"resp_share_jeune_{k}")
        if f"reason_jeune_{k}" in out.columns:     final.append(f"reason_jeune_{k}")
        if f"age_jeune_{k}" in out.columns:        final.append(f"age_jeune_{k}")
    out = out[final].sort_values("Num_Acc").reset_index(drop=True)

    out.to_csv(out_csv, index=False, encoding="utf-8")
    return out


In [24]:
compute_young_driver_responsability_strict(
    baac_csv="result20_25.csv",
    out_csv="p_resp_young_drivers_per_accident.csv",
    out_csv_drivers="p_resp_par_conducteur.csv",
    age_min=20,
    age_max=25
)

,Num_Acc,nb_conducteurs_total,nb_conducteurs_jeunes,pct_conducteurs_jeunes,resp_share_jeunes_total,resp_share_autres_total,resp_share_total_check,p_resp_moyenne_jeunes,resp_share_jeune_1,reason_jeune_1,age_jeune_1,resp_share_jeune_2,reason_jeune_2,age_jeune_2,resp_share_jeune_3,reason_jeune_3,age_jeune_3,resp_share_jeune_4,reason_jeune_4,age_jeune_4
0,202300000005,2,2,1.0,1.000000,0.000000,1.0,0.141851,0.500000,col:3(+1.0);obsm:2(+0.5);vma:30(+0.2);lum:5(+0.3),22.0,0.500000,col:3(+1.0);obsm:2(+0.5);vma:30(+0.2);lum:5(+0.3),22.0,NaN,NaN,NaN,NaN,NaN,NaN
1,202300000009,2,1,0.5,0.561314,0.438686,1.0,0.201009,0.561314,col:3(+1.0);choc:1(+0.5);obsm:2(+0.5);catr:3(+...,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202300000010,1,1,1.0,1.000000,0.000000,1.0,0.201009,1.000000,choc:1(+0.5);obsm:1(+1.5);catr:3(+0.5);vma:50(...,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202300000012,2,1,0.5,0.500000,0.500000,1.0,0.157095,0.500000,col:3(+1.0);obsm:2(+0.5);catr:1(+0.5);vma:50(+...,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202300000015,2,2,1.0,1.000000,0.000000,1.0,0.348286,0.548762,col:2(+2.0);choc:4(+1.0);obsm:2(+0.5);catr:3(+...,24.0,0.451238,col:2(+2.0);choc:1(+0.5);obsm:2(+0.5);catr:3(+...,22.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14240,202300054809,2,1,0.5,0.641648,0.358352,1.0,0.659260,0.641648,manv:11(+2.0);col:3(+1.0);choc:7(+0.5);obsm:2(...,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14241,202300054812,2,2,1.0,1.000000,0.000000,1.0,0.354344,0.500000,col:3(+1.0);choc:8(+0.5);obsm:2(+0.5);catr:1(+...,25.0,0.500000,col:3(+1.0);choc:7(+0.5);obsm:2(+0.5);catr:1(+...,22.0,NaN,NaN,NaN,NaN,NaN,NaN
14242,202300054818,1,1,1.0,1.000000,0.000000,1.0,0.083173,1.000000,choc:8(+0.5);vma:50(+0.2);lum:5(+0.3),21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14243,202300054819,2,1,0.5,0.414755,0.585245,1.0,0.354344,0.414755,manv:21(+1.0);col:3(+1.0);choc:8(+0.5);obsm:2(...,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
import pandas as pd
import numpy as np

def build_per_accident_matrix(baac_csv, out_csv, age_min=20, age_max=25):

    df = pd.read_csv(baac_csv, low_memory=False)
    df.columns = [c.strip().replace(" ","_") for c in df.columns]


    grav_map = {1:"Indemne",4:"Blessé léger",3:"Blessé hospitalisé",2:"Tué"}
    df["grav_label"] = df["grav"].map(grav_map).fillna("NA")


    victim_counts = (df
        .groupby(["Num_Acc","grav_label"], dropna=False)
        .size()
        .unstack(fill_value=0)
        .reset_index()
    )

    catu_counts = (df
        .groupby(["Num_Acc","catu"], dropna=False)
        .size()
        .unstack(fill_value=0)
        .add_prefix("catu_")
        .reset_index()
    )

    veh_agg = (df
        .groupby("Num_Acc", as_index=False)
        .agg(
            nb_veh=("id_vehicule", pd.Series.nunique),
            catr_mode=("catr", lambda x: x.mode().iloc[0] if len(x.mode()) else np.nan),
            vma_mode=("vma", lambda x: pd.Series(x).mode().iloc[0] if len(pd.Series(x).mode()) else np.nan),
            col_mode=("col", lambda x: x.mode().iloc[0] if len(x.mode()) else np.nan),
            choc_mode=("choc", lambda x: x.mode().iloc[0] if len(x.mode()) else np.nan),
            manv_mode=("manv", lambda x: x.mode().iloc[0] if len(x.mode()) else np.nan),
            obs_mode=("obs", lambda x: x.mode().iloc[0] if len(x.mode()) else np.nan),
            obsm_mode=("obsm", lambda x: x.mode().iloc[0] if len(x.mode()) else np.nan),
            agg_mode=("agg", lambda x: x.mode().iloc[0] if len(x.mode()) else np.nan),
            lum_mode=("lum", lambda x: x.mode().iloc[0] if len(x.mode()) else np.nan),
        )
    )

    victim_summary = df.groupby("Num_Acc", as_index=False).agg(
        victimes_total=("id_usager","count"),
        victimes_corporelles=("grav", lambda s: int((s!=1).sum())),
        blesses_legers=("grav", lambda s: int((s==4).sum())),
        blesses_hosp=("grav", lambda s: int((s==3).sum())),
        deces=("grav", lambda s: int((s==2).sum())),
    )

    catv_counts = (df.groupby(["Num_Acc","catv"]).size().unstack(fill_value=0))
    catv_counts = catv_counts.add_prefix("catv_").reset_index()

    acc = (victim_counts
           .merge(catu_counts, on="Num_Acc", how="outer")
           .merge(veh_agg, on="Num_Acc", how="outer")
           .merge(victim_summary, on="Num_Acc", how="outer")
           .merge(catv_counts, on="Num_Acc", how="outer")
          )

    count_cols = [c for c in acc.columns if c not in [
        "Num_Acc","catr_mode","vma_mode","col_mode",
        "choc_mode","manv_mode","obs_mode","obsm_mode",
        "agg_mode","lum_mode"
    ]]
    acc[count_cols] = acc[count_cols].fillna(0)


    try:
        resp_acc = compute_young_driver_responsability_strict(
            baac_csv=baac_csv,
            out_csv="__tmp_accident_level.csv",
            out_csv_drivers="__tmp_drivers.csv",
            age_min=age_min, age_max=age_max
        )
    except NameError:
        resp_acc = compute_young_driver_responsability_strict(
            baac_csv=baac_csv,
            out_csv="__tmp_accident_level.csv",
            age_min=age_min, age_max=age_max
        )

    if resp_acc is None or resp_acc.empty:
        acc["resp_share_jeunes_total"] = np.nan
        acc["resp_share_jeune_max"] = np.nan
        acc.to_csv(out_csv, index=False, encoding="utf-8")
        return acc

    jeune_cols = [c for c in resp_acc.columns if c.startswith("resp_share_jeune_")]
    resp_acc = resp_acc.copy()
    resp_acc["resp_share_jeune_max"] = resp_acc[jeune_cols].max(axis=1) if len(jeune_cols) else np.nan

    resp_keep = resp_acc[["Num_Acc","resp_share_jeunes_total","resp_share_jeune_max"]].copy()


    acc["Num_Acc"] = acc["Num_Acc"].astype(str).str.strip()
    resp_keep["Num_Acc"] = resp_keep["Num_Acc"].astype(str).str.strip()

    acc = acc.merge(resp_keep, on="Num_Acc", how="left")


    acc.to_csv(out_csv, index=False, encoding="utf-8")
    return acc


In [26]:
#2023
if __name__ == "__main__":
    build_per_accident_matrix("result20_25.csv", "result20_25matrix.csv")


In [27]:
def _pick_step_mult(steps, v):
    for thr, m in steps:
        if v <= thr:
            return m
    return steps[-1][1]

def _safe_int(x, default=0):
    try:
        return int(x)
    except Exception:
        return default

def _safe_float(x, default=0.0):
    try:
        return float(x)
    except Exception:
        return default

def estimer_couts_ligne(row):
    n_leger = _safe_int(row.get("blesses_legers", 0))
    n_grave = _safe_int(row.get("blesses_hosp", 0))
    n_deces = _safe_int(row.get("deces", 0))
    cout_corp_base = n_leger * COUT_CORPO["leger"] + n_grave * COUT_CORPO["grave"] + n_deces * COUT_CORPO["deces"]

    lum = _safe_int(row.get("lum_mode", 1), 1)
    agg = _safe_int(row.get("agg_mode", 2), 2)
    catr = _safe_int(row.get("catr_mode", 4), 4)
    vma = _safe_float(row.get("vma_mode", 50), 50.0)
    manv = _safe_int(row.get("manv_mode", 0))
    obsm = _safe_int(row.get("obsm_mode", 0))
    obs = _safe_int(row.get("obs_mode", 0))

    mul_corp = 1.0
    mul_corp *= LUM_CORP_MULT.get(lum, 1.0)
    mul_corp *= AGG_CORP_MULT.get(agg, 1.0)
    mul_corp *= CATR_CORP_MULT.get(catr, 1.0)
    mul_corp *= _pick_step_mult(VMA_CORP_MULT, vma)
    mul_corp *= MANV_CORP_MULT.get(manv, 1.0)
    mul_corp *= OBSM_CORP_MULT.get(obsm, 1.0)
    mul_corp *= OBS_CORP_MULT.get(obs, 1.0)

    for col in [c for c in row.index if str(c).startswith("catv_")]:
        if row[col] and _safe_int(row[col], 0) > 0:
            code = _safe_int(col.split("_")[1], 0)
            mul_corp *= CATV_CORP_VULN_BONUS.get(code, 1.0)

    if _safe_int(row.get("catu_3", 0), 0) > 0:
        mul_corp *= PEDESTRIAN_CORP_BONUS

    cout_corp = cout_corp_base * mul_corp

    cout_mat_base = 0.0
    for col in [c for c in row.index if str(c).startswith("catv_")]:
        qty = _safe_int(row[col], 0)
        if qty > 0:
            code = _safe_int(col.split("_")[1], 0)
            base = COUT_MAT_CATV.get(code, DEFAULT_COUT_MAT_CATV)
            cout_mat_base += qty * base

    col = _safe_int(row.get("col_mode", 6), 6)
    choc = _safe_int(row.get("choc_mode", 0), 0)

    mul_mat = 1.0
    mul_mat *= COL_MAT_MULT.get(col, 1.0)
    mul_mat *= CHOC_MAT_MULT.get(choc, 1.0)
    mul_mat *= _pick_step_mult(VMA_MAT_MULT, vma)
    mul_mat *= AGG_MAT_MULT.get(agg, 1.0)

    cout_mat = cout_mat_base * mul_mat

    cout_total = cout_corp + cout_mat
    resp_share_jeunes = _safe_float(row.get("resp_share_jeunes_total", 0.0), 0.0)
    resp_share_jeune_max = _safe_float(row.get("resp_share_jeune_max", 0.0), 0.0)

    cout_jeunes_total = cout_total * max(0.0, min(1.0, resp_share_jeunes))
    cout_jeune_max = cout_total * max(0.0, min(1.0, resp_share_jeune_max))

    return pd.Series({
        "cout_corp_base": round(cout_corp_base, 2),
        "mul_corp": round(mul_corp, 4),
        "cout_corp": round(cout_corp, 2),
        "cout_mat_base": round(cout_mat_base, 2),
        "mul_mat": round(mul_mat, 4),
        "cout_mat": round(cout_mat, 2),
        "cout_total": round(cout_total, 2),
        "cout_jeunes_total": round(cout_jeunes_total, 2),
        "cout_jeune_max": round(cout_jeune_max, 2)
    })

def appliquer_estimations_matrice(matrice_csv: str, out_csv: str = "matrix_with_costs_full.csv") -> pd.DataFrame:
    df = pd.read_csv(matrice_csv, low_memory=False)
    df.columns = [c.strip().replace(" ", "_") for c in df.columns]
    est = df.apply(estimer_couts_ligne, axis=1)
    df_out = pd.concat([df, est], axis=1)
    df_out.to_csv(out_csv, index=False, encoding="utf-8")
    return df_out

In [28]:
if __name__ == "__main__":
    appliquer_estimations_matrice("result20_25matrix.csv", "result20_25matrix_with_costs_full.csv")

In [29]:
finalmatrix = pd.read_csv('result20_25matrix_with_costs_full.csv')

In [31]:
finalmatrix.cout_jeunes_total.sum()

np.float64(950753164.81)

In [45]:
finalmatrix.cout_jeunes_total.mean()

np.float64(66742.9389126009)

In [47]:
finalmatrix.cout_total.mean()

np.float64(97091.16117234118)